<a href="https://colab.research.google.com/github/JRCon1/GreeksPackage-Beta-/blob/main/Greeks_Package.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install py_vollib

from py_vollib.black_scholes.greeks.analytical import delta, gamma, vega, theta, rho
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta
from scipy.stats import norm
import warnings
import yfinance as yf
import pandas as pd
from datetime import datetime

# Suppress all warnings (optional, but not recommended for debugging)
warnings.simplefilter(action='ignore', category=FutureWarning)

def download_options(
    ticker_symbol,
    opt_type='c',
    max_days=60,
    lower_moneyness=0.95,
    upper_moneyness=1.05,
    price=False  # New optional parameter
):
    """
    Downloads and filters option chains for a given ticker according to:
      1. Option type (calls or puts)
      2. Maximum days to expiration
      3. Moneyness bounds
      4. Optionally includes the stock price in each row (useful for ITM/OTM visualization).

    Parameters:
        ticker_symbol (str): The stock ticker.
        opt_type (str, optional): 'c' for calls, 'p' for puts (default: 'c').
        max_days (int, optional): Max days until expiration (default: 60).
        lower_moneyness (float, optional): Lower bound for moneyness (default: 0.95).
        upper_moneyness (float, optional): Upper bound for moneyness (default: 1.05).
        price (bool, optional): If True, adds a 'Stock Price' column with the current stock price.

    Returns:
        pd.DataFrame: Filtered options chain.
    """

    # Retrieve the ticker data from yfinance
    ticker = yf.Ticker(ticker_symbol)

    # Grab the current underlying price
    underlying_price = ticker.history(period="1d")['Close'].iloc[-1]

    # Calculate the strike range using the specified moneyness
    lower_strike = underlying_price * lower_moneyness
    upper_strike = underlying_price * upper_moneyness

    # Prepare a DataFrame to hold all filtered data
    relevant_columns = [
        'contractSymbol',
        'inTheMoney',
        'strike',
        'lastPrice',
        'bid',
        'ask',
        'volume',
        'openInterest',
        'impliedVolatility'
    ]
    filtered_options = pd.DataFrame(columns=relevant_columns + ['expiry'])

    # Loop through each available expiration date, filtering by max_days
    for expiry_date_str in ticker.options:
        expiry_date = pd.to_datetime(expiry_date_str)
        days_to_expiry = (expiry_date - datetime.now()).days

        if days_to_expiry <= max_days:
            # Retrieve calls or puts for the given expiration
            option_chain = ticker.option_chain(expiry_date_str)
            if opt_type.lower() == 'c':
                data = option_chain.calls
            elif opt_type.lower() == 'p':
                data = option_chain.puts
            else:
                continue

            # Filter by strike based on moneyness
            data = data[(data['strike'] >= lower_strike) & (data['strike'] <= upper_strike)].copy()

            # Attach an expiry column
            data['expiry'] = expiry_date

            # Concatenate only if data is non-empty
            if not data.empty:
                data = data[relevant_columns + ['expiry']]
                filtered_options = pd.concat([filtered_options, data], ignore_index=True)

    # Calculate Days to Expiry for each row
    filtered_options['Days to Expiry'] = (
        pd.to_datetime(filtered_options['expiry']) - datetime.now()
    ).dt.days

    # Calculate a Mid-Point price from bid and ask
    filtered_options['Mid-Point Price'] = round((filtered_options['bid'] + filtered_options['ask']) / 2, 4)

    filtered_options['impliedVolatility'] = filtered_options['impliedVolatility'].round(2)

    # If include_stock_price is True, add a 'Stock Price' column
    if price:
        filtered_options['Stock Price'] = round(underlying_price, 4)

    return filtered_options

filtered_options = download_options('AAPL', opt_type='c', max_days=60, lower_moneyness=0.95, upper_moneyness=1.05, price = True)

In [ ]:
#D1 & D2
def compute_d1(S, K, t, r, sigma, epsilon=1e-9):
    t = max(t, epsilon)  # This line and the following need to be indented
    return (np.log(S / K) + (r + 0.5 * sigma ** 2) * t) / (sigma * np.sqrt(t))

def compute_d2(S, K, t, r, sigma, epsilon=1e-9):
    return compute_d1(S, K, t, r, sigma, epsilon) - sigma * np.sqrt(t)

def compute_d1_d2(S, K, t, r, sigma, epsilon=1e-9):
    t = max(t, epsilon)
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * t) / (sigma * np.sqrt(t))
    return d1, d1 - sigma * np.sqrt(t)

In [ ]:
import numpy as np
import yfinance as yf
import pandas as pd
from scipy.stats import norm

def vanna(row: pd.Series, ticker: str, r: float = 0.05, option_type: str = 'c', epsilon: float = 1e-9) -> float:
    """
    Computes Vanna (sensitivity of Vega to volatility) for a single row of an options DataFrame.

    Vanna measures the rate of change of Vega with respect to changes in the underlying price.
    It is important in managing risk when trading volatility-sensitive strategies.

    Parameters:
        row (pd.Series): A row from the options DataFrame containing:
            - 'strike' (float): Strike price of the option.
            - 'impliedVolatility' (float): Annualized implied volatility (decimal).
            - 'Days to Expiry' (int): Number of days until expiration.
        ticker (str): Ticker symbol for the underlying stock.
        r (float, optional): Risk-free interest rate (default 0.05).
        option_type (str, optional): Option type, either call ('c') or put ('p') (default 'c').
        epsilon (float, optional): Small value to avoid division by zero errors (default 1e-9).

    Returns:
        pd.Series: Contains Vanna, d1, and d2 values.

    Formula:
        Vanna = e^(-rT) * N'(d1) * (d2 / sigma)

    Notes:
        - This function retrieves the latest stock price using Yahoo Finance.
        - Uses the Black-Scholes model assumptions.
        - Assumes continuous compounding for risk-free rate.
    """
    # Fetch latest stock price
    S = yf.Ticker(ticker).history(period="1d")['Close'].iloc[-1]

    # Extract option details
    K = row['strike']  # Strike price
    T = max(row['Days to Expiry'] / 365, epsilon)  # Ensure T > 0
    sigma = max(row['impliedVolatility'], 0.01)  # Prevent division by zero

    # Validate inputs
    if pd.isna(S) or pd.isna(K) or pd.isna(sigma) or S <= 0 or sigma <= 0:
        return pd.Series({'Vanna': float('nan'), 'd1': float('nan'), 'd2': float('nan')})

    # Compute d1 and d2
    d1, d2 = compute_d1_d2(S, K, T, r, sigma)

    # Compute normal PDF of d1
    N_prime_d1 = norm.pdf(d1)

    # Compute Vanna using the correct formula
    vanna_val = np.exp(-r * T) * N_prime_d1 * (d2 / sigma)

    return round(vanna_val, 4)


In [ ]:
import numpy as np
import yfinance as yf
import pandas as pd
from scipy.stats import norm

def volga(row: pd.Series, ticker: str, r: float = 0.05, option_type: str = 'c', epsilon: float = 1e-9) -> float:
    """
    Computes Volga (sensitivity of Vega to volatility) for a single row of an options DataFrame.

    Volga measures how Vega changes with respect to volatility. It helps quantify how sensitive
    an option's price is to second-order volatility effects.

    Parameters:
        row (pd.Series): A row from the options DataFrame containing:
            - 'strike' (float): Strike price of the option.
            - 'impliedVolatility' (float): Annualized implied volatility (decimal).
            - 'Days to Expiry' (int): Number of days until expiration.
        ticker (str): Ticker symbol for the underlying stock.
        r (float, optional): Risk-free interest rate (default 0.05).
        option_type (str, optional): Option type, either call ('c') or put ('p') (default 'c').
        epsilon (float, optional): Small value to avoid division by zero errors (default 1e-9).

    Returns:
        float: Computed Volga value rounded to four decimal places.

    Formula:
        Volga = Vega * (d1 * d2) / sigma

    Notes:
        - This function retrieves the latest stock price using Yahoo Finance.
        - Uses the Black-Scholes model assumptions.
        - Assumes continuous compounding for risk-free rate.
    """
    # Fetch latest stock price
    S = yf.Ticker(ticker).history(period="1d")['Close'].iloc[-1]

    # Extract option details
    K = row['strike']  # Strike price
    T = max(row['Days to Expiry'] / 365, epsilon)  # Ensure T > 0
    sigma = max(row['impliedVolatility'], 0.01)  # Prevent division by zero

    # Compute d1 and d2
    d1, d2 = compute_d1_d2(S, K, T, r, sigma)

    # Compute Vega using py_vollib or a custom implementation
    vega_val = vega(option_type, S, K, T, r, sigma)

    # Compute Volga using the refined formula
    volga_val = vega_val * (d1 * d2) / sigma

    return round(volga_val, 4)

In [ ]:
import numpy as np
import yfinance as yf
import pandas as pd
from scipy.stats import norm

def charm(row: pd.Series, ticker: str, r: float = 0.05, option_type: str = 'c', epsilon: float = 1e-9) -> float:
    """
    Computes Charm (Delta decay) for a single row of an options DataFrame.

    Charm measures how Delta changes with respect to time, representing the rate at which
    Delta decays as expiration approaches. This is crucial for managing directional risk in options trading.

    Parameters:
        row (pd.Series): A row from the options DataFrame containing:
            - 'strike' (float): Strike price of the option.
            - 'impliedVolatility' (float): Annualized implied volatility (decimal).
            - 'Days to Expiry' (int): Number of days until expiration.
        ticker (str): Ticker symbol for the underlying stock.
        r (float, optional): Risk-free interest rate (default 0.05).
        option_type (str, optional): Option type, either call ('c') or put ('p') (default 'c').
        epsilon (float, optional): Small value to avoid division by zero errors (default 1e-9).

    Returns:
        float: Computed Charm value rounded to four decimal places.

    Formula:
        Charm = -N'(d1) * (2 * r * T - d2 * sigma * sqrt(T)) / (2 * T * sigma * sqrt(T))

    Notes:
        - This function retrieves the latest stock price using Yahoo Finance.
        - Uses the Black-Scholes model assumptions.
        - Assumes continuous compounding for risk-free rate.
    """
    # Fetch latest stock price
    S = yf.Ticker(ticker).history(period="1d")['Close'].iloc[-1]

    # Extract option details
    K = row['strike']  # Strike price
    T = max(row['Days to Expiry'] / 365, epsilon)  # Ensure T > 0
    sigma = max(row['impliedVolatility'], 0.01)  # Prevent division by zero

    # Compute d1 and d2
    d1, d2 = compute_d1_d2(S, K, T, r, sigma)

    # Compute normal PDF of d1
    N_prime_d1 = norm.pdf(d1)  # Equivalent to (1 / sqrt(2π)) * exp(-0.5 * d1^2)

    # Compute Charm using the correct formula
    charm_value = -N_prime_d1 * (2 * r * T - d2 * sigma * np.sqrt(T)) / (2 * T)

    return round(charm_value, 4)

In [ ]:
import numpy as np
import yfinance as yf
import pandas as pd
from scipy.stats import norm

def veta(row: pd.Series, ticker: str, r: float = 0.05, option_type: str = 'c', epsilon: float = 1e-9) -> float:
    """
    Computes Veta (sensitivity of Vega to time decay) for a single row of an options DataFrame.

    Veta measures the rate at which Vega changes as time passes, indicating how an option's sensitivity
    to volatility changes with time.

    Parameters:
        row (pd.Series): A row from the options DataFrame containing:
            - 'strike' (float): Strike price of the option.
            - 'impliedVolatility' (float): Annualized implied volatility (decimal).
            - 'Days to Expiry' (int): Number of days until expiration.
        ticker (str): Ticker symbol for the underlying stock.
        r (float, optional): Risk-free interest rate (default 0.05).
        option_type (str, optional): Option type, either call ('c') or put ('p') (default 'c').
        epsilon (float, optional): Small value to avoid division by zero errors (default 1e-9).

    Returns:
        float: Computed Veta value rounded to four decimal places.

    Formula:
        Veta = vega_val * np.exp(-r * T) * norm.pdf(d1) * np.sqrt(T) * (r - d1 / (2 * T)) / sigma

    Notes:
        - This function retrieves the latest stock price using Yahoo Finance.
        - Uses the Black-Scholes model assumptions.
        - Assumes continuous compounding for risk-free rate.
    """
    # Fetch latest stock price
    S = yf.Ticker(ticker).history(period="1d")['Close'].iloc[-1]
    K = row['strike']
    T = max(row['Days to Expiry'] / 365, epsilon)
    sigma = max(row['impliedVolatility'], 0.01)

    d1, d2 = compute_d1_d2(S, K, T, r, sigma)
    vega_val = vega(option_type, S, K, T, r, sigma)

    # More detailed Veta calculation incorporating interest rate changes
    interest_rate_term = (r - d1 / (2 * T)) / sigma
    veta_val = vega_val * np.exp(-r * T) * norm.pdf(d1) * np.sqrt(T) * interest_rate_term

    return round(veta_val, 4)

**THIRD DERIVATIVES**

In [ ]:
import numpy as np
import yfinance as yf
import pandas as pd
from scipy.stats import norm

def color(row: pd.Series, ticker: str, r: float = 0.05, option_type: str = 'c', epsilon: float = 1e-9) -> float:
    """
    Computes Color (Gamma decay) for a single row of an options DataFrame.

    Color measures how Gamma changes as time passes, indicating how the convexity of an option’s
    Delta shifts with time. It is crucial for managing second-order sensitivity to underlying price changes.

    Parameters:
        row (pd.Series): A row from the options DataFrame containing:
            - 'strike' (float): Strike price of the option.
            - 'impliedVolatility' (float): Annualized implied volatility (decimal).
            - 'Days to Expiry' (int): Number of days until expiration.
        ticker (str): Ticker symbol for the underlying stock.
        r (float, optional): Risk-free interest rate (default 0.05).
        option_type (str, optional): Option type, either call ('c') or put ('p') (default 'c').
        epsilon (float, optional): Small value to avoid division by zero errors (default 1e-9).

    Returns:
        float: Computed Color value rounded to four decimal places.

    Formula:
        Color = (N'(d1) / (2 * S * T * sigma * sqrt(T))) * (2 * r * T + 1 - d1 * d2)

    Notes:
        - This function retrieves the latest stock price using Yahoo Finance.
        - Uses the Black-Scholes model assumptions.
        - Assumes continuous compounding for risk-free rate.
    """
    # Fetch latest stock price
    S = yf.Ticker(ticker).history(period="1d")['Close'].iloc[-1]

    # Extract option details
    K = row['strike']  # Strike price
    T = max(row['Days to Expiry'] / 365, epsilon)  # Ensure T > 0
    sigma = max(row['impliedVolatility'], 0.01)  # Prevent division by zero

    # Compute d1 and d2
    d1, d2 = compute_d1_d2(S, K, T, r, sigma)

    # Compute normal PDF of d1
    N_prime_d1 = norm.pdf(d1)  # Equivalent to (1 / sqrt(2π)) * exp(-0.5 * d1^2)

    # Compute Color using refined formula
    color_value = (N_prime_d1 / (2 * S * T * sigma * np.sqrt(T))) * (2 * r * T + 1 - d1 * d2)

    return round(color_value, 4)


In [ ]:
import numpy as np
import yfinance as yf
import pandas as pd
from scipy.stats import norm

def speed(row: pd.Series, ticker: str, r: float = 0.05, option_type: str = 'c', epsilon: float = 1e-9) -> float:
    """
    Computes Speed (rate of change of Gamma) for a single row of an options DataFrame.

    Speed measures how rapidly Gamma changes as the underlying price moves, representing
    the third derivative of the option price with respect to the stock price. It helps
    assess how stable or unstable an option's Delta and Gamma are.

    Parameters:
        row (pd.Series): A row from the options DataFrame containing:
            - 'strike' (float): Strike price of the option.
            - 'impliedVolatility' (float): Annualized implied volatility (decimal).
            - 'Days to Expiry' (int): Number of days until expiration.
        ticker (str): Ticker symbol for the underlying stock.
        r (float, optional): Risk-free interest rate (default 0.05).
        option_type (str, optional): Option type, either call ('c') or put ('p') (default 'c').
        epsilon (float, optional): Small value to avoid division by zero errors (default 1e-9).

    Returns:
        float: Computed Speed value rounded to four decimal places.

    Formula:
        Speed = (N'(d1) / (S^2 * sigma * sqrt(T))) * (d1 / (sigma * sqrt(T)) - 1)

    Notes:
        - This function retrieves the latest stock price using Yahoo Finance.
        - Uses the Black-Scholes model assumptions.
        - Assumes continuous compounding for risk-free rate.
    """
    # Fetch latest stock price
    S = yf.Ticker(ticker).history(period="1d")['Close'].iloc[-1]

    # Extract option details
    K = row['strike']  # Strike price
    T = max(row['Days to Expiry'] / 365, epsilon)  # Ensure T > 0
    sigma = max(row['impliedVolatility'], 0.01)  # Prevent division by zero

    # Compute d1 and d2
    d1, d2 = compute_d1_d2(S, K, T, r, sigma)

    # Compute normal PDF of d1
    N_prime_d1 = norm.pdf(d1)  # Equivalent to (1 / sqrt(2π)) * exp(-0.5 * d1^2)

    # Compute Speed using the correct formula
    speed_value = (N_prime_d1 / (S ** 2 * sigma * np.sqrt(T))) * ((d1 / (sigma * np.sqrt(T))) - 1)

    return round(speed_value, 4)

In [ ]:
import numpy as np
import yfinance as yf
import pandas as pd
from scipy.stats import norm

def ultima(row: pd.Series, ticker: str, r: float = 0.05, option_type: str = 'c', epsilon: float = 1e-9) -> float:
    """
    Computes Ultima (sensitivity of Vanna to volatility) for a single row of an options DataFrame.

    Ultima measures the rate of change of Vanna with respect to changes in implied volatility.
    It helps quantify how much the convexity of Vega changes as volatility shifts.

    Parameters:
        row (pd.Series): A row from the options DataFrame containing:
            - 'strike' (float): Strike price of the option.
            - 'impliedVolatility' (float): Annualized implied volatility (decimal).
            - 'Days to Expiry' (int): Number of days until expiration.
        ticker (str): Ticker symbol for the underlying stock.
        r (float, optional): Risk-free interest rate (default 0.05).
        option_type (str, optional): Option type, either call ('c') or put ('p') (default 'c').
        epsilon (float, optional): Small value to avoid division by zero errors (default 1e-9).

    Returns:
        float: Computed Ultima value rounded to four decimal places.

    Formula:
        Ultima = (Vega * (d1 * d2 - 1) * d1 * d2) / sigma

    Notes:
        - This function retrieves the latest stock price using Yahoo Finance.
        - Uses the Black-Scholes model assumptions.
        - Assumes continuous compounding for risk-free rate.
    """
    # Fetch latest stock price
    S = yf.Ticker(ticker).history(period="1d")['Close'].iloc[-1]

    # Extract option details
    K = row['strike']  # Strike price
    T = max(row['Days to Expiry'] / 365, epsilon)  # Ensure T > 0
    sigma = max(row['impliedVolatility'], 0.01)  # Prevent division by zero

    # Compute d1 and d2
    d1, d2 = compute_d1_d2(S, K, T, r, sigma)

    # Compute Vega using py_vollib
    vega_val = vega(option_type, S, K, T, r, sigma)

    # Compute Ultima using refined formula
    ultima_value = (vega_val * (d1 * d2 - 1) * d1 * d2) / sigma

    return round(ultima_value, 4)


In [ ]:
import numpy as np
import yfinance as yf
import pandas as pd
from scipy.stats import norm

def zomma(row: pd.Series, ticker: str, r: float = 0.05, option_type: str = 'c', epsilon: float = 1e-9) -> float:
    """
    Computes Zomma (sensitivity of Gamma to volatility) for a single row of an options DataFrame.

    Zomma measures the rate of change of Gamma with respect to changes in implied volatility.
    It helps quantify how much Gamma fluctuates when volatility changes.

    Parameters:
        row (pd.Series): A row from the options DataFrame containing:
            - 'strike' (float): Strike price of the option.
            - 'impliedVolatility' (float): Annualized implied volatility (decimal).
            - 'Days to Expiry' (int): Number of days until expiration.
        ticker (str): Ticker symbol for the underlying stock.
        r (float, optional): Risk-free interest rate (default 0.05).
        option_type (str, optional): Option type, either call ('c') or put ('p') (default 'c').
        epsilon (float, optional): Small value to avoid division by zero errors (default 1e-9).

    Returns:
        float: Computed Zomma value rounded to four decimal places.

    Formula:
        Zomma = (Gamma * (d1 * d2 - 1)) / sigma

    Notes:
        - This function retrieves the latest stock price using Yahoo Finance.
        - Uses the Black-Scholes model assumptions.
        - Assumes continuous compounding for risk-free rate.
    """
    # Fetch latest stock price
    S = yf.Ticker(ticker).history(period="1d")['Close'].iloc[-1]

    # Extract option details
    K = row['strike']  # Strike price
    T = max(row['Days to Expiry'] / 365, epsilon)  # Ensure T > 0
    sigma = max(row['impliedVolatility'], 0.01)  # Prevent division by zero

    # Compute d1 and d2
    d1, d2 = compute_d1_d2(S, K, T, r, sigma)

    # Compute Gamma using py_vollib
    gamma_val = gamma(option_type, S, K, T, r, sigma)

    # Compute Zomma using refined formula
    zomma_value = (gamma_val * (d1 * d2 - 1)) / sigma

    return round(zomma_value, 4)


Primary Functions

In [ ]:
def first_order(row: pd.Series, ticker: str, r: float = 0.05, option_type: str = 'c', epsilon: float = 1e-9) -> pd.Series:
    """
    Computes all first-order Greeks (Delta, Vega, Theta, Rho)
    for a single row of an options DataFrame.

    This function calculates and returns the relevant first-order Greeks in a single call.

    Parameters:
        row (pd.Series): A row from the options DataFrame containing:
            - 'strike' (float): Strike price of the option.
            - 'impliedVolatility' (float): Annualized implied volatility (decimal).
            - 'Days to Expiry' (int): Number of days until expiration.
        ticker (str): Ticker symbol for the underlying stock.
        r (float, optional): Risk-free interest rate (default 0.05).
        option_type (str, optional): Option type, either call ('c') or put ('p') (default 'c').
        epsilon (float, optional): Small value to avoid division by zero errors (default 1e-9).

    Returns:
        pd.Series: A Pandas Series containing all first-order Greeks.

    Notes:
        - This function retrieves the latest stock price using Yahoo Finance.
        - Uses the Black-Scholes model assumptions.
        - Assumes continuous compounding for risk-free rate.
    """

    # Fetch latest stock price
    try:
        S = yf.Ticker(ticker).history(period="1d")['Close'].iloc[-1]
    except:
        return pd.Series({'Error': 'Stock price retrieval failed'})

    # Extract option details
    K = row['strike']
    T = max(row['Days to Expiry'] / 365, epsilon)  # Ensure T > 0
    sigma = max(row['impliedVolatility'], 0.01)  # Prevent division by zero

    # Validate inputs
    if pd.isna(S) or pd.isna(K) or pd.isna(sigma) or S <= 0 or sigma <= 0:
        return pd.Series({'Delta': float('nan'), 'Vega': float('nan'), 'Theta': float('nan'), 'Rho': float('nan')})

    # Compute first-order Greeks using py_vollib
    greek_values = {
        'Delta': delta(option_type, S, K, T, r, sigma),
        'Vega': vega(option_type, S, K, T, r, sigma),
        'Theta': theta(option_type, S, K, T, r, sigma),
        'Rho': rho(option_type, S, K, T, r, sigma)
    }

    return pd.Series({key: round(value, 4) for key, value in greek_values.items()})

In [ ]:
def second_order(row: pd.Series, ticker: str, r: float = 0.05, option_type: str = 'c', epsilon: float = 1e-9) -> pd.Series:
    """
    Computes all second-order Greeks (Vanna, Volga, Veta, Charm, Gamma)
    for a single row of an options DataFrame.

    This function calculates and returns the relevant second-order Greeks in a single call.

    Parameters:
        row (pd.Series): A row from the options DataFrame containing:
            - 'strike' (float): Strike price of the option.
            - 'impliedVolatility' (float): Annualized implied volatility (decimal).
            - 'Days to Expiry' (int): Number of days until expiration.
        ticker (str): Ticker symbol for the underlying stock.
        r (float, optional): Risk-free interest rate (default 0.05).
        option_type (str, optional): Option type, either call ('c') or put ('p') (default 'c').
        epsilon (float, optional): Small value to avoid division by zero errors (default 1e-9).

    Returns:
        pd.Series: A Pandas Series containing all second-order Greeks.

    Notes:
        - This function retrieves the latest stock price using Yahoo Finance.
        - Uses the Black-Scholes model assumptions.
        - Assumes continuous compounding for risk-free rate.
    """
    # Fetch latest stock price
    try:
        S = yf.Ticker(ticker).history(period="1d")['Close'].iloc[-1]
    except:
        return pd.Series({'Error': 'Stock price retrieval failed'})

    # Extract option details
    K = row['strike']
    T = max(row['Days to Expiry'] / 365, epsilon)  # Ensure T > 0
    sigma = max(row['impliedVolatility'], 0.01)  # Prevent division by zero

    # Validate inputs
    if pd.isna(S) or pd.isna(K) or pd.isna(sigma) or S <= 0 or sigma <= 0:
        return pd.Series({'Vanna': float('nan'), 'Volga': float('nan'), 'Veta': float('nan'),
                          'Charm': float('nan'), 'Gamma': float('nan')})

    # Compute d1 and d2 once
    d1, d2 = compute_d1_d2(S, K, T, r, sigma)

    # Compute all second-order Greeks
    greek_values = {
        'Vanna': vanna(row, ticker, r, option_type),
        'Volga': volga(row, ticker, r, option_type),
        'Veta': veta(row, ticker, r, option_type),
        'Charm': charm(row, ticker, r, option_type),
        'Gamma': gamma(option_type, S, K, T, r, sigma)  # Gamma from py_vollib
    }

    return pd.Series({key: round(value, 4) for key, value in greek_values.items()})

In [ ]:
def third_order(row: pd.Series, ticker: str, r: float = 0.05, option_type: str = 'c', epsilon: float = 1e-9) -> pd.Series:
    """
    Computes all third-order Greeks (Color, Speed, Ultima, Zomma)
    for a single row of an options DataFrame.

    This function calculates and returns all relevant third-order Greeks in a single call.

    Parameters:
        row (pd.Series): A row from the options DataFrame containing:
            - 'strike' (float): Strike price of the option.
            - 'impliedVolatility' (float): Annualized implied volatility (decimal).
            - 'Days to Expiry' (int): Number of days until expiration.
        ticker (str): Ticker symbol for the underlying stock.
        r (float, optional): Risk-free interest rate (default 0.05).
        option_type (str, optional): Option type, either call ('c') or put ('p') (default 'c').
        epsilon (float, optional): Small value to avoid division by zero errors (default 1e-9).

    Returns:
        pd.Series: A Pandas Series containing all third-order Greeks.

    Notes:
        - This function retrieves the latest stock price using Yahoo Finance.
        - Uses the Black-Scholes model assumptions.
        - Assumes continuous compounding for risk-free rate.
    """
    # Fetch latest stock price
    try:
        S = yf.Ticker(ticker).history(period="1d")['Close'].iloc[-1]
    except:
        return pd.Series({'Error': 'Stock price retrieval failed'})

    # Extract option details
    K = row['strike']
    T = max(row['Days to Expiry'] / 365, epsilon)  # Ensure T > 0
    sigma = max(row['impliedVolatility'], 0.01)  # Prevent division by zero

    # Validate inputs
    if pd.isna(S) or pd.isna(K) or pd.isna(sigma) or S <= 0 or sigma <= 0:
        return pd.Series({'Color': float('nan'), 'Speed': float('nan'), 'Ultima': float('nan'), 'Zomma': float('nan')})

    # Compute d1 and d2 once
    d1, d2 = compute_d1_d2(S, K, T, r, sigma)

    # Compute all third-order Greeks
    greek_values = {
        'Color': color(row, ticker, r, option_type),
        'Speed': speed(row, ticker, r, option_type),
        'Ultima': ultima(row, ticker, r, option_type),
        'Zomma': zomma(row, ticker, r, option_type)
    }

    return pd.Series({key: round(value, 4) for key, value in greek_values.items()})

In [ ]:
def greeks(row: pd.Series, ticker: str, r: float = 0.05, option_type: str = 'c', epsilon: float = 1e-9) -> pd.Series:
    """
    Computes all Greeks (First-order, Second-order, and Third-order)
    for a single row of an options DataFrame.

    This function calculates and returns all relevant Greeks in a single call.

    Parameters:
        row (pd.Series): A row from the options DataFrame containing:
            - 'strike' (float): Strike price of the option.
            - 'impliedVolatility' (float): Annualized implied volatility (decimal).
            - 'Days to Expiry' (int): Number of days until expiration.
        ticker (str): Ticker symbol for the underlying stock.
        r (float, optional): Risk-free interest rate (default 0.05).
        option_type (str, optional): Option type, either call ('c') or put ('p') (default 'c').
        epsilon (float, optional): Small value to avoid division by zero errors (default 1e-9).

    Returns:
        pd.Series: A Pandas Series containing all Greeks.

    Notes:
        - This function retrieves the latest stock price using Yahoo Finance.
        - Uses the Black-Scholes model assumptions.
        - Assumes continuous compounding for risk-free rate.
    """
    # Fetch latest stock price
    try:
        S = yf.Ticker(ticker).history(period="1d")['Close'].iloc[-1]
    except:
        return pd.Series({'Error': 'Stock price retrieval failed'})

    # Extract option details
    K = row['strike']
    T = max(row['Days to Expiry'] / 365, epsilon)  # Ensure T > 0
    sigma = max(row['impliedVolatility'], 0.01)  # Prevent division by zero

    # Validate inputs
    if pd.isna(S) or pd.isna(K) or pd.isna(sigma) or S <= 0 or sigma <= 0:
        return pd.Series({
            'Delta': float('nan'), 'Vega': float('nan'), 'Theta': float('nan'), 'Rho': float('nan'),
            'Gamma': float('nan'), 'Vanna': float('nan'), 'Volga': float('nan'), 'Veta': float('nan'), 'Charm': float('nan'),
            'Color': float('nan'), 'Speed': float('nan'), 'Ultima': float('nan'), 'Zomma': float('nan')
        })

    # Compute d1 and d2 once
    d1, d2 = compute_d1_d2(S, K, T, r, sigma)

    # Compute First-Order Greeks using py_vollib
    first_order_values = {
        'Delta': delta(option_type, S, K, T, r, sigma),
        'Vega': vega(option_type, S, K, T, r, sigma),
        'Theta': theta(option_type, S, K, T, r, sigma),
        'Rho': rho(option_type, S, K, T, r, sigma),
    }

    # Compute Second-Order Greeks
    second_order_values = {
        'Gamma': gamma(option_type, S, K, T, r, sigma),
        'Vanna': vanna(row, ticker, r, option_type),
        'Volga': volga(row, ticker, r, option_type),
        'Veta': veta(row, ticker, r, option_type),
        'Charm': charm(row, ticker, r, option_type),
    }

    # Compute Third-Order Greeks
    third_order_values = {
        'Color': color(row, ticker, r, option_type),
        'Speed': speed(row, ticker, r, option_type),
        'Ultima': ultima(row, ticker, r, option_type),
        'Zomma': zomma(row, ticker, r, option_type),
    }

    # Combine all Greeks into a single dictionary
    all_greeks = {**first_order_values, **second_order_values, **third_order_values}

    # Ensure no NoneType values exist (convert None to NaN)
    all_greeks = {key: (round(value, 4) if value is not None else float('nan')) for key, value in all_greeks.items()}

    return pd.Series(all_greeks)


filtered_options[['Delta', 'Vega', 'Theta', 'Rho',
                  'Gamma', 'Vanna', 'Volga', 'Veta', 'Charm',
                  'Color', 'Speed', 'Ultima', 'Zomma']] = \
    filtered_options.apply(lambda row: greeks(row, ticker='AAPL'), axis=1)

In [ ]:
filtered_options

,contractSymbol,inTheMoney,strike,lastPrice,bid,ask,volume,openInterest,impliedVolatility,expiry,...,Rho,Gamma,Vanna,Volga,Veta,Charm,Color,Speed,Ultima,Zomma
0,AAPL250214C00217500,True,217.5,10.85,10.55,10.90,109,253,0.32,2025-02-14,...,0.0264,0.0214,0.6909,0.2307,-0.1474,0.2933,-0.4057,0.0030,0.1204,0.0348
1,AAPL250214C00220000,True,220.0,8.60,8.40,8.65,1016,1201,0.29,2025-02-14,...,0.0254,0.0300,0.8053,0.2236,-0.2194,0.2779,-0.0529,0.0039,0.0111,0.0051
2,AAPL250214C00222500,True,222.5,6.44,6.40,6.60,323,409,0.28,2025-02-14,...,0.0231,0.0409,0.7620,0.1489,-0.2764,0.2402,0.7287,0.0038,-0.0725,-0.0711
3,AAPL250214C00225000,True,225.0,4.70,4.65,4.75,1927,2546,0.26,2025-02-14,...,0.0201,0.0527,0.5465,0.0611,-0.2442,0.1397,1.6133,0.0030,-0.0511,-0.1697
4,AAPL250214C00227500,True,227.5,3.20,3.10,3.25,5082,1435,0.25,2025-02-14,...,0.0159,0.0598,0.0451,0.0007,-0.0406,-0.0079,2.1820,0.0003,-0.0007,-0.2388
5,AAPL250214C00230000,False,230.0,2.00,1.96,2.04,20188,5000,0.24,2025-02-14,...,0.0113,0.0591,-0.5637,0.0496,0.2243,-0.1577,1.9041,-0.0033,-0.0438,-0.2170
6,AAPL250214C00232500,False,232.5,1.14,1.14,1.17,21852,6192,0.24,2025-02-14,...,0.0073,0.0483,-0.9560,0.1823,0.3241,-0.2509,0.8285,-0.0056,-0.0854,-0.0943
7,AAPL250214C00235000,False,235.0,0.64,0.63,0.64,29139,12261,0.23,2025-02-14,...,0.0039,0.0338,-1.0547,0.3219,0.2433,-0.2489,-0.4195,-0.0065,0.1099,0.0502
8,AAPL250214C00237500,False,237.5,0.36,0.35,0.36,10604,6269,0.24,2025-02-14,...,0.0022,0.0211,-0.8429,0.3309,0.1271,-0.2143,-0.9307,-0.0050,0.4005,0.1064
9,AAPL250221C00217500,True,217.5,11.75,11.30,11.55,124,3911,0.28,2025-02-21,...,0.0583,0.0219,0.8151,0.3216,-0.2470,0.1639,0.0469,0.0017,-0.0442,-0.0108
